In [1]:
import urllib
import requests
from bs4 import BeautifulSoup
import pandas as pd
url = 'https://www.charleroi-airport.com/en/flights/timetable/index.html'

In [2]:
page = urllib.request.urlopen(url)

In [3]:
soup = BeautifulSoup(page, 'html.parser')

# Function definitions

In [4]:
def parse_table(table):
    col_names = [col.text for col in table.find_all('thead')[0].find_all("th")]
    flight = table.previous_sibling.previous_sibling.h2.string
    content = list()
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        row = list(td.string for td in tds)
        if len(row):
            content.append(row)
    return flight, col_names, content

def shorten_flight(flight):
    new_flight = flight[5:]  # It starts always with 'From '
    new_flight = new_flight.replace("Brussels South Charleroi", "CRL").replace(" to ", "->")
    return new_flight

def generate_df(tables):
    dfs = list()
    for table in tables:
        flight, col_names, content = parse_table(table)
        flight = shorten_flight(flight)
#         print(flight)
        index = pd.MultiIndex.from_arrays([[flight]*len(content), list(range(len(content)))], names=("connection", "#"))
        df = pd.DataFrame(content, columns=col_names, index=index)
        df['Mon.'] = df['Mon.'].astype(bool)
        df.loc[:, 'Mon.':'Sun.'] = df.loc[:, 'Mon.':'Sun.'].applymap(bool) #.astype(bool) # I have to do this after having explicitly converted (only) one column otherwise it does not work
        df['Start'] = pd.to_datetime(df['Start'])
        df['End'] = pd.to_datetime(df['End'])
        dfs.append(df)
#     assert len(dfs) == 2 * len(cities) +1, "Cities are  cities than flights"
    return pd.concat(dfs)

## Debug

In [5]:
table0 = soup.find_all('table')[0]
table0

<table class="timeTable">
<thead>
<tr>
<th>Flightnr.</th>
<th>Depart</th>
<th>Arrive</th>
<th>Start</th>
<th>End</th>
<th>Mon.</th>
<th>Tue.</th>
<th>Wed.</th>
<th>Thu.</th>
<th>Fri.</th>
<th>Sat.</th>
<th>Sun.</th>
</tr>
</thead>
<tbody>
<tr class="impair">
<td>FR8172</td><td>06:30</td>
<td>09:20</td>
<td>30/10/2017</td>
<td>22/12/2017</td>
<td><div align="center">X</div></td>
<td></td>
<td><div align="center">X</div></td>
<td></td>
<td><div align="center">X</div></td>
<td></td>
<td></td>
</tr>
<tr class="pair">
<td>FR8172</td><td>06:30</td>
<td>09:20</td>
<td>19/12/2017</td>
<td>19/12/2017</td>
<td></td>
<td><div align="center">X</div></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr class="impair">
<td>FR8172</td><td>06:30</td>
<td>09:20</td>
<td>27/12/2017</td>
<td>05/01/2018</td>
<td></td>
<td></td>
<td><div align="center">X</div></td>
<td></td>
<td><div align="center">X</div></td>
<td></td>
<td></td>
</tr>
<tr class="pair">
<td>FR8172</td><td>06:30</td>
<td>09:20<

Take the first row to get the column names

In [6]:
column_names = [col.text for col in table0.find_all('thead')[0].find_all("th")]
column_names

['Flightnr.',
 'Depart',
 'Arrive',
 'Start',
 'End',
 'Mon.',
 'Tue.',
 'Wed.',
 'Thu.',
 'Fri.',
 'Sat.',
 'Sun.']

In [7]:
parse_table(table0)

('From Brussels South Charleroi to Agadir',
 ['Flightnr.',
  'Depart',
  'Arrive',
  'Start',
  'End',
  'Mon.',
  'Tue.',
  'Wed.',
  'Thu.',
  'Fri.',
  'Sat.',
  'Sun.'],
 [['FR8172',
   '06:30',
   '09:20',
   '30/10/2017',
   '22/12/2017',
   'X',
   None,
   'X',
   None,
   'X',
   None,
   None],
  ['FR8172',
   '06:30',
   '09:20',
   '19/12/2017',
   '19/12/2017',
   None,
   'X',
   None,
   None,
   None,
   None,
   None],
  ['FR8172',
   '06:30',
   '09:20',
   '27/12/2017',
   '05/01/2018',
   None,
   None,
   'X',
   None,
   'X',
   None,
   None],
  ['FR8172',
   '06:30',
   '09:20',
   '02/01/2018',
   '02/01/2018',
   None,
   'X',
   None,
   None,
   None,
   None,
   None],
  ['FR8172',
   '06:30',
   '09:20',
   '08/01/2018',
   '23/03/2018',
   'X',
   None,
   'X',
   None,
   'X',
   None,
   None],
  ['FR8172',
   '08:00',
   '10:50',
   '01/01/2018',
   '01/01/2018',
   'X',
   None,
   None,
   None,
   None,
   None,
   None],
  ['FR8172',
   '13:50',
  

In [8]:
flight, col_names, content = parse_table(table0)
content

[['FR8172',
  '06:30',
  '09:20',
  '30/10/2017',
  '22/12/2017',
  'X',
  None,
  'X',
  None,
  'X',
  None,
  None],
 ['FR8172',
  '06:30',
  '09:20',
  '19/12/2017',
  '19/12/2017',
  None,
  'X',
  None,
  None,
  None,
  None,
  None],
 ['FR8172',
  '06:30',
  '09:20',
  '27/12/2017',
  '05/01/2018',
  None,
  None,
  'X',
  None,
  'X',
  None,
  None],
 ['FR8172',
  '06:30',
  '09:20',
  '02/01/2018',
  '02/01/2018',
  None,
  'X',
  None,
  None,
  None,
  None,
  None],
 ['FR8172',
  '06:30',
  '09:20',
  '08/01/2018',
  '23/03/2018',
  'X',
  None,
  'X',
  None,
  'X',
  None,
  None],
 ['FR8172',
  '08:00',
  '10:50',
  '01/01/2018',
  '01/01/2018',
  'X',
  None,
  None,
  None,
  None,
  None,
  None],
 ['FR8172',
  '13:50',
  '16:40',
  '31/10/2017',
  '31/10/2017',
  None,
  'X',
  None,
  None,
  None,
  None,
  None],
 ['FR8172',
  '14:50',
  '17:40',
  '21/12/2017',
  '04/01/2018',
  None,
  None,
  None,
  'X',
  None,
  None,
  None]]

Create the dataframe

In [9]:
df = pd.DataFrame(content, columns=col_names)
df

,Flightnr.,Depart,Arrive,Start,End,Mon.,Tue.,Wed.,Thu.,Fri.,Sat.,Sun.
0,FR8172,06:30,09:20,30/10/2017,22/12/2017,X,None,X,None,X,None,None
1,FR8172,06:30,09:20,19/12/2017,19/12/2017,None,X,None,None,None,None,None
2,FR8172,06:30,09:20,27/12/2017,05/01/2018,None,None,X,None,X,None,None
3,FR8172,06:30,09:20,02/01/2018,02/01/2018,None,X,None,None,None,None,None
4,FR8172,06:30,09:20,08/01/2018,23/03/2018,X,None,X,None,X,None,None
5,FR8172,08:00,10:50,01/01/2018,01/01/2018,X,None,None,None,None,None,None
6,FR8172,13:50,16:40,31/10/2017,31/10/2017,None,X,None,None,None,None,None
7,FR8172,14:50,17:40,21/12/2017,04/01/2018,None,None,None,X,None,None,None


Change the types of some columns

In [10]:
df['Mon.'] = df['Mon.'].astype(bool)
df.loc[:, 'Mon.':'Sun.'] = df.loc[:, 'Mon.':'Sun.'].applymap(bool) #.astype(bool) # I have to do this after having explicitly converted (only) one column otherwise it does not work
df['Start'] = pd.to_datetime(df['Start'])
df['End'] = pd.to_datetime(df['End'])
df

,Flightnr.,Depart,Arrive,Start,End,Mon.,Tue.,Wed.,Thu.,Fri.,Sat.,Sun.
0,FR8172,06:30,09:20,2017-10-30,2017-12-22,True,False,True,False,True,False,False
1,FR8172,06:30,09:20,2017-12-19,2017-12-19,False,True,False,False,False,False,False
2,FR8172,06:30,09:20,2017-12-27,2018-05-01,False,False,True,False,True,False,False
3,FR8172,06:30,09:20,2018-02-01,2018-02-01,False,True,False,False,False,False,False
4,FR8172,06:30,09:20,2018-08-01,2018-03-23,True,False,True,False,True,False,False
5,FR8172,08:00,10:50,2018-01-01,2018-01-01,True,False,False,False,False,False,False
6,FR8172,13:50,16:40,2017-10-31,2017-10-31,False,True,False,False,False,False,False
7,FR8172,14:50,17:40,2017-12-21,2018-04-01,False,False,False,True,False,False,False


In [11]:
df.dtypes

Flightnr.            object
Depart               object
Arrive               object
Start        datetime64[ns]
End          datetime64[ns]
Mon.                   bool
Tue.                   bool
Wed.                   bool
Thu.                   bool
Fri.                   bool
Sat.                   bool
Sun.                   bool
dtype: object

Shorten the flight name

In [12]:
shorten_flight("From Brussels South Charleroi to Palma de Majorca")

'CRL->Palma de Majorca'

Create an index and apply it

In [13]:
index = pd.MultiIndex.from_arrays([[shorten_flight(flight)]*len(content), list(range(len(content)))], names=("connection", "#"))

In [14]:
df.index = index
df

Flightnr. Depart Arrive      Start        End   Mon.   Tue.  \
connection  #                                                               
CRL->Agadir 0    FR8172  06:30  09:20 2017-10-30 2017-12-22   True  False   
            1    FR8172  06:30  09:20 2017-12-19 2017-12-19  False   True   
            2    FR8172  06:30  09:20 2017-12-27 2018-05-01  False  False   
            3    FR8172  06:30  09:20 2018-02-01 2018-02-01  False   True   
            4    FR8172  06:30  09:20 2018-08-01 2018-03-23   True  False   
            5    FR8172  08:00  10:50 2018-01-01 2018-01-01   True  False   
            6    FR8172  13:50  16:40 2017-10-31 2017-10-31  False   True   
            7    FR8172  14:50  17:40 2017-12-21 2018-04-01  False  False   

                Wed.   Thu.   Fri.   Sat.   Sun.  
connection  #                                     
CRL->Agadir 0   True  False   True  False  False  
            1  False  False  False  False  False  
            2   True  False   True  False  False  
            3  False  False  False  False  False  
            4   True  False   True  False  False  
            5  False  False  False  False  False  
            6  False  False  False  False  False  
            7  False   True  False  False  False

# Find cities

In [15]:
l = soup.find_all("a")
cities = list()
for c in l:
    if c.string is not None and c.string.lower() != 'top' and "#" in c['href']:
        cities.append(c.string)
cities[:5], len(cities)

(['Agadir', 'Ajaccio', 'Algiers', 'Alghero', 'Alicante'], 91)

# Create the dataframe

In [16]:
tables = soup.find_all('table')

Be aware that the last table is not a flight

In [17]:
tables[-1]

<table width="80%">
<tbody><tr>
<td><input class="icon-textbox login" id="memberformuser" name="user" size="20" type="text" value="a@a.org"/></td>
</tr>
<tr>
<td><input class="icon-textbox password" id="memberformpass" name="pass" size="20" type="password" value="password"/></td>
</tr>
<tr>
<td align="center" height="40" valign="bottom">
<input class="button" id="memberformsubmit" name="submit" type="button" value="Connexion"/>
</td>
</tr>
<tr>
<td colspan="2" id="memberformloginfailed"></td>
</tr>
</tbody></table>

In [18]:
df_tot = generate_df(tables[:-1])

In [19]:
df_tot.tail()

Flightnr. Depart Arrive      Start        End   Mon.   Tue.  \
connection    #                                                               
CRL->Zaragoza 1    FR1382  08:00  10:00 2018-02-01 2018-02-01  False   True   
              2    FR1382  17:00  19:00 2017-01-11 2018-03-24  False  False   
Zaragoza->CRL 0    FR1383  10:25  12:30 2017-12-19 2017-12-19  False   True   
              1    FR1383  10:25  12:30 2018-02-01 2018-02-01  False   True   
              2    FR1383  19:25  21:30 2017-01-11 2018-03-24  False  False   

                  Wed.   Thu.   Fri.   Sat.   Sun.  
connection    #                                     
CRL->Zaragoza 1  False  False  False  False  False  
              2   True  False  False   True  False  
Zaragoza->CRL 0  False  False  False  False  False  
              1  False  False  False  False  False  
              2   True  False  False   True  False

Putting all together

In [20]:
def parse_flights(url='https://www.charleroi-airport.com/en/flights/timetable/index.html'):
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    tables = soup.find_all('table')
    return generate_df(tables[:-1])

In [21]:
df_flights = parse_flights()

In [22]:
departures = df_flights[df_flights.index.get_level_values('connection').str.startswith('CRL')]
arrivals = df_flights[df_flights.index.get_level_values('connection').str.endswith('CRL')]

In [23]:
df_flights.index.get_level_values('connection')

Index(['CRL->Agadir', 'CRL->Agadir', 'CRL->Agadir', 'CRL->Agadir',
       'CRL->Agadir', 'CRL->Agadir', 'CRL->Agadir', 'CRL->Agadir',
       'Agadir->CRL', 'Agadir->CRL',
       ...
       'Vilnius->CRL', 'Vilnius->CRL', 'CRL->Zadar', 'Zadar->CRL',
       'CRL->Zaragoza', 'CRL->Zaragoza', 'CRL->Zaragoza', 'Zaragoza->CRL',
       'Zaragoza->CRL', 'Zaragoza->CRL'],
      dtype='object', name='connection', length=1242)

In [24]:
df_flights["Arrive"].max()

'23:30'